In [7]:
from random import randint, seed

dimension = 4
perimeter = 50
sensitivity = 1
jsonfilename = "matrices4x4.json"

%run generate_matrices_tools.ipynb

In [10]:
generate_new_examples(10000, dimension, perimeter, False, False)

TypeError: generate_new_examples() takes from 1 to 3 positional arguments but 5 were given

# Generating new examples

This notebook's purpose is to generate new examples of *dysfunctional matrices* - matrices for which the cube algorithm gives different result than the LLL algorithm. For now, this code generates only square matrices. It returns a json file containing the *dysfunctional* matrices and their core parameters.

The generating function takes in parameters above.

`dimension`: the shape of the matrix

`perimeter`: defines an interval `[- perimeter, perimeter]` for the integers inside the matrix

`sensitivity`: lol i forgot

`jsonfilename`: name of the ouput file


In [5]:
# How many matrices are dysfunctional?

iterations = 2000
dim = 100
perimeter = 200
print("dimension: ", dim)
dysfunctional = generate_new_examples(iterations, dim, perimeter, printing=False, functioning=False)
print("There's {} dysfunctional matrices of dimension {} out of total {} instances, percent: {}". format(dysfunctional, dim, iterations, float(100*dysfunctional/iterations)))


dimension:  100


KeyboardInterrupt: 